In [23]:
import pandas as pd
from pathlib import Path
Path.cwd()

In [41]:
frame = pd.read_csv('resources/survey_results_public.csv', index_col="Respondent")
pd.set_option('display.max_columns', 85)


In [42]:
# frame.shape
# frame.info()
frame.head()

,MainBranch,Hobbyist,OpenSourcer,OpenSource,Employment,Country,Student,EdLevel,UndergradMajor,EduOther,OrgSize,DevType,YearsCode,Age1stCode,YearsCodePro,CareerSat,JobSat,MgrIdiot,MgrMoney,MgrWant,JobSeek,LastHireDate,LastInt,FizzBuzz,JobFactors,ResumeUpdate,CurrencySymbol,CurrencyDesc,CompTotal,CompFreq,ConvertedComp,WorkWeekHrs,WorkPlan,WorkChallenge,WorkRemote,WorkLoc,ImpSyn,CodeRev,CodeRevHrs,UnitTests,PurchaseHow,PurchaseWhat,LanguageWorkedWith,LanguageDesireNextYear,DatabaseWorkedWith,DatabaseDesireNextYear,PlatformWorkedWith,PlatformDesireNextYear,WebFrameWorkedWith,WebFrameDesireNextYear,MiscTechWorkedWith,MiscTechDesireNextYear,DevEnviron,OpSys,Containers,BlockchainOrg,BlockchainIs,BetterLife,ITperson,OffOn,SocialMedia,Extraversion,ScreenName,SOVisit1st,SOVisitFreq,SOVisitTo,SOFindAnswer,SOTimeSaved,SOHowMuchTime,SOAccount,SOPartFreq,SOJobs,EntTeams,SOComm,WelcomeChange,SONewContent,Age,Gender,Trans,Sexuality,Ethnicity,Dependents,SurveyLength,SurveyEase
Respondent,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,I am a student who is learning to code,Yes,Never,The quality of OSS and closed source software ...,"Not employed, and not looking for work",United Kingdom,No,Primary/elementary school,NaN,"Taught yourself a new language, framework, or ...",NaN,NaN,4,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HTML/CSS;Java;JavaScript;Python,C;C++;C#;Go;HTML/CSS;Java;JavaScript;Python;SQL,SQLite,MySQL,MacOS;Windows,Android;Arduino;Windows,Django;Flask,Flask;jQuery,Node.js,Node.js,IntelliJ;Notepad++;PyCharm,Windows,I do not use containers,NaN,NaN,Yes,"Fortunately, someone else has that title",Yes,Twitter,Online,Username,2017,A few times per month or weekly,Find answers to specific questions;Learn how t...,3-5 times per week,Stack Overflow was much faster,31-60 minutes,No,NaN,"No, I didn't know that Stack Overflow had a jo...","No, and I don't know what those are",Neutral,Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,14.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
2,I am a student who is learning to code,No,Less than once per year,The quality of OSS and closed source software ...,"Not employed, but looking for work",Bosnia and Herzegovina,"Yes, full-time","Secondary school (e.g. American high school, G...",NaN,Taken an online course in programming or softw...,NaN,"Developer, desktop or enterprise applications;...",NaN,17,NaN,NaN,NaN,NaN,NaN,NaN,I am actively looking for a job,I've never had a job,NaN,NaN,Financial performance or funding status of the...,"Something else changed (education, award, medi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C++;HTML/CSS;Python,C++;HTML/CSS;JavaScript;SQL,NaN,MySQL,Windows,Windows,Django,Django,NaN,NaN,Atom;PyCharm,Windows,I do not use containers,NaN,Useful across many domains and could change ma...,Yes,Yes,Yes,Instagram,Online,Username,2017,Daily or almost daily,Find answers to specific questions;Learn how t...,3-5 times per week,Stack Overflow was much faster,11-30 minutes,Yes,A few times per month or weekly,"No, I knew that Stack Overflow had a job board...","No, and I don't know what those are","Yes, somewhat",Just as welcome now as I felt last year,Tech articles written by other developers;Indu...,19.0,Man,No,Straight / Heterosexual,NaN,No,Appropriate in length,Neither easy nor difficult
3,"I am not primarily a developer, but I write co...",Yes,Never,The quality of OSS and closed source software ...,Employed full-time,Thailand,No,"Bachelor’s degree (BA, BS, B.Eng., etc.)",Web development or web design,"Taught yourself a new language, framework, or ...",100 to 499 employees,"Designer;Developer, back-end;Developer, front-...",3,22,1,Slightly satisfied,Slightly satisfied,Not at all confident,Not sure,Not sure,"I’m not actively looking, but I am open to new...",1-2 years ago,Int

In [28]:
schema_frame = pd.read_csv('resources/survey_results_schema.csv', index_col="Column")
pd.set_option('display.max_rows', 85)

In [34]:
schema_frame.shape
schema_frame.info()
schema_frame.head()

<class 'pandas.core.frame.DataFrame'>
Index: 85 entries, Respondent to SurveyEase
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   QuestionText  85 non-null     object
dtypes: object(1)
memory usage: 3.4+ KB


,QuestionText
Column,
Respondent,Randomized respondent ID number (not in order ...
MainBranch,Which of the following options best describes ...
Hobbyist,Do you code as a hobby?
OpenSourcer,How often do you contribute to open source?
OpenSource,How do you feel about the quality of open sour...


In [38]:
# Now we can search on `Column` column with loc function, because the default index is changed.
# (first parameter is for row filter aand second one is for column name)
schema_frame.loc["Respondent", "QuestionText"] 

'Randomized respondent ID number (not in order of survey response time)'

In [ ]:
# What is a dataframe?
# They are a simple schema over a table with concists of rows and columns
# It is also something like a dictionary with list items BUT with a large number of added functionalities.
# DataFrames are rows and columns and Series are the rows of the columns.

In [ ]:
frame[["Respondent", "Hobbyist"]]

In [ ]:
frame.iloc[1000:1010] # Integer location of a row

In [ ]:
frame.iloc[[1000,1001, 1002], [1,2]] # Integer location of the rows and special columns

In [ ]:
frame.loc[1000:1005, ["MainBranch","Hobbyist"]] # Integer location (as deafault indexes) of the rows and special columns using loc

In [ ]:
frame["Hobbyist"].value_counts()

In [ ]:
frame.loc[1000:1005, "MainBranch":"Country"] # Define column range

In [40]:
# Indexes are not forced to be unique for pandas
# But it would have better performance if we create it on unique columns
# Indexes are labels for rows
# frame.set_index("Respondent", inplace=True) # With inplace=True set the current index as a default index
# frame.index
# frame
# frame.loc[1] # Now it is using `Respondent` columns as index and search in this column for value `1`
# frame.reset_index(inplace=True)
# schema_frame.sort_index(ascending=False, inplace=True) # inplace=True makes this descending order for index permanent

In [119]:
countries_filter = frame["Country"].isin(["United States", "Iran", "Germany"])
age_filter = frame["Age1stCode"] == "29"
pythonist_filter = frame["LanguageWorkedWith"].str.contains("Python", na=False) == True # na=False means exclude rows with NaN from filtering to avoid exceptions
# frame[countries_filter & age_filter & pythonist_filter]
filtered = frame.loc[
    countries_filter & age_filter & pythonist_filter,
    ("LanguageWorkedWith", "Student", "Age", "Country")
] # Preferred way

# filtered.shape, filtered.size, len(filtered)
filtered 

,LanguageWorkedWith,Student,Age,Country
Respondent,,,,
4197,Bash/Shell/PowerShell;C;HTML/CSS;Java;JavaScri...,No,36.0,United States
5961,Assembly;Bash/Shell/PowerShell;C++;HTML/CSS;Ja...,"Yes, part-time",32.0,United States
6064,Assembly;Bash/Shell/PowerShell;C;C++;C#;Dart;G...,No,29.0,United States
10103,Bash/Shell/PowerShell;JavaScript;Python;R;SQL,No,39.0,United States
13558,HTML/CSS;JavaScript;PHP;Python;Swift;TypeScrip...,No,48.0,United States
20772,C#;HTML/CSS;JavaScript;Python;SQL;TypeScript,"Yes, full-time",31.0,United States
22189,HTML/CSS;Java;Python;Other(s):,No,38.0,United States
23568,Bash/Shell/PowerShell;C#;Python;SQL,No,38.0,United States
25405,Python;SQL,No,30.0,Germany
